# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, project_tests
%autoreload 1

2024-10-08 09:22:49.151818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 09:22:49.151875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 09:22:49.151935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 09:22:49.163776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Should return keras==2.14.0 and tf-keras==2.14.1
!pip freeze | grep keras

keras==2.14.0
tf-keras==2.14.1


In [3]:
# Should return tensorflow==2.14.0
!pip freeze | grep tensorflow

tensorflow==2.14.0
tensorflow-datasets==4.9.4
tensorflow-estimator==2.14.0
tensorflow-hub==0.16.1
tensorflow-io-gcs-filesystem==0.34.0
tensorflow-metadata==1.15.0


In [16]:
import collections

import helper
import numpy as np
import project_tests as tests

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # Initialize tokenizer
    tokenizer = Tokenizer()

    # Fit tokenizer 
    tokenizer.fit_on_texts(x)

    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # Find the length of the longest sequence
    if length is None:
        length = max([len(seq) for seq in x])
    
    # Pad the sequences
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [20]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3

    # Define the input layer
    inputs = Input(shape=input_shape[1:])
    
    # Build the layers
    gru = GRU(128, return_sequences=True)(inputs)
    time_dist = TimeDistributed(Dense(french_vocab_size))(gru)
    outputs = Activation('softmax')(time_dist)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 4s 19ms/step - loss: 2.9719 - accuracy: 0.4406 - val_loss: nan - val_accuracy: 0.4861
Epoch 2/10
108/108 [==============================] - 1s 14ms/step - loss: 2.1177 - accuracy: 0.5207 - val_loss: nan - val_accuracy: 0.5674
Epoch 3/10
108/108 [==============================] - 1s 13ms/step - loss: 1.7236 - accuracy: 0.5811 - val_loss: nan - val_accuracy: 0.5907
Epoch 4/10
108/108 [==============================] - 1s 14ms/step - loss: 1.5544 - accuracy: 0.5950 - val_loss: nan - val_accuracy: 0.6059
Epoch 5/10
108/108 [==============================] - 1s 14ms/step - loss: 1.4568 - accuracy: 0.6134 - val_loss: nan - val_accuracy: 0.6163
Epoch 6/10
108/108 [==============================] - 1s 14ms/step - loss: 1.3789 - accuracy: 0.6266 - val_loss: nan - val_accuracy: 0.6315
Epoch 7/10
108/108 [==============================] - 1s 14ms/step - loss: 1.3122 - accuracy: 0.6374 - val_loss: nan - val_accuracy: 0.6426
Epoch 8/10
108/108 [

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [22]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3

    # Define the input layer
    inputs = Input(shape=input_shape[1:])

    # Add embedding layer
    embedding = Embedding(english_vocab_size, 128, input_length=input_shape[1])(inputs)

    # Build the layers
    gru = GRU(128, return_sequences=True)(embedding)
    time_dist = TimeDistributed(Dense(french_vocab_size))(gru)
    outputs = Activation('softmax')(time_dist)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train the neural network

embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 7s 40ms/step - loss: 3.3279 - accuracy: 0.4148 - val_loss: nan - val_accuracy: 0.4770
Epoch 2/10
108/108 [==============================] - 2s 20ms/step - loss: 2.1007 - accuracy: 0.5474 - val_loss: nan - val_accuracy: 0.6255
Epoch 3/10
108/108 [==============================] - 2s 16ms/step - loss: 1.2806 - accuracy: 0.6898 - val_loss: nan - val_accuracy: 0.7437
Epoch 4/10
108/108 [==============================] - 2s 15ms/step - loss: 0.9267 - accuracy: 0.7665 - val_loss: nan - val_accuracy: 0.7862
Epoch 5/10
108/108 [==============================] - 2s 15ms/step - loss: 0.7272 - accuracy: 0.8028 - val_loss: nan - val_accuracy: 0.8183
Epoch 6/10
108/108 [==============================] - 2s 17ms/step - loss: 0.6011 - accuracy: 0.8291 - val_loss: nan - val_accuracy: 0.8403
Epoch 7/10
108/108 [==============================] - 2s 15ms/step - loss: 0.5190 - accuracy: 0.8479 - val_loss: nan - val_accuracy: 0.8578
Epoch 8/10
108/108 [

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [23]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3

    # Define the input layer
    inputs = Input(shape=input_shape[1:])

    # Add bidirectional layer
    bd = Bidirectional(GRU(128, return_sequences=True))(inputs)
    time_dist = TimeDistributed(Dense(french_vocab_size))(bd)
    outputs = Activation('softmax')(time_dist)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)


# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))


bd_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 7s 29ms/step - loss: 2.5729 - accuracy: 0.5111 - val_loss: nan - val_accuracy: 0.5726
Epoch 2/10
108/108 [==============================] - 2s 21ms/step - loss: 1.5587 - accuracy: 0.6015 - val_loss: nan - val_accuracy: 0.6195
Epoch 3/10
108/108 [==============================] - 2s 21ms/step - loss: 1.3540 - accuracy: 0.6288 - val_loss: nan - val_accuracy: 0.6424
Epoch 4/10
108/108 [==============================] - 2s 21ms/step - loss: 1.2393 - accuracy: 0.6512 - val_loss: nan - val_accuracy: 0.6593
Epoch 5/10
108/108 [==============================] - 2s 21ms/step - loss: 1.1619 - accuracy: 0.6653 - val_loss: nan - val_accuracy: 0.6706
Epoch 6/10
108/108 [==============================] - 2s 21ms/step - loss: 1.1036 - accuracy: 0.6758 - val_loss: nan - val_accuracy: 0.6798
Epoch 7/10
108/108 [==============================] - 2s 21ms/step - loss: 1.0568 - accuracy: 0.6835 - val_loss: nan - val_accuracy: 0.6873
Epoch 8/10
108/108 [

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [24]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3

    # Define the input layer
    inputs = Input(shape=input_shape[1:])

    # Add encoder-decoder model layers
    encoder = GRU(128)(inputs)
    encoder = RepeatVector(output_sequence_length)(encoder)
    decoder = GRU(128, return_sequences=True)(encoder)
    time_dist = TimeDistributed(Dense(french_vocab_size))(decoder)
    outputs = Activation('softmax')(time_dist)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 8s 29ms/step - loss: 3.1294 - accuracy: 0.4320 - val_loss: nan - val_accuracy: 0.4729
Epoch 2/10
108/108 [==============================] - 2s 21ms/step - loss: 2.3824 - accuracy: 0.4909 - val_loss: nan - val_accuracy: 0.5044
Epoch 3/10
108/108 [==============================] - 2s 20ms/step - loss: 2.2094 - accuracy: 0.5039 - val_loss: nan - val_accuracy: 0.5200
Epoch 4/10
108/108 [==============================] - 2s 20ms/step - loss: 2.0961 - accuracy: 0.5286 - val_loss: nan - val_accuracy: 0.5438
Epoch 5/10
108/108 [==============================] - 2s 21ms/step - loss: 1.9118 - accuracy: 0.5447 - val_loss: nan - val_accuracy: 0.5497
Epoch 6/10
108/108 [==============================] - 2s 21ms/step - loss: 1.7281 - accuracy: 0.5549 - val_loss: nan - val_accuracy: 0.5653
Epoch 7/10
108/108 [==============================] - 2s 21ms/step - loss: 1.6333 - accuracy: 0.5725 - val_loss: nan - val_accuracy: 0.5804
Epoch 8/10
108/108 [

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [26]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Learning rate
    learning_rate = 0.005

    # Define the input layer
    inputs = Input(shape=input_shape[1:])

    # Add embedding, encoder-decoder, and bidirectional layers
    embedding = Embedding(english_vocab_size, 128, input_length=input_shape[1])(inputs)
    bd = Bidirectional(GRU(128, return_sequences=False))(embedding)
    encoder = RepeatVector(output_sequence_length)(bd)
    decoder = Bidirectional(GRU(128, return_sequences=True))(encoder)
    time_dist = TimeDistributed(Dense(french_vocab_size))(decoder)
    outputs = Activation('softmax')(time_dist)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

# Train the final model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)


final_rnn_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

final_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Epoch 1/10
108/108 [==============================] - 16s 71ms/step - loss: 2.2073 - accuracy: 0.5087 - val_loss: nan - val_accuracy: 0.6245
Epoch 2/10
108/108 [==============================] - 5s 47ms/step - loss: 1.2208 - accuracy: 0.6687 - val_loss: nan - val_accuracy: 0.7265
Epoch 3/10
108/108 [==============================] - 5s 46ms/step - loss: 0.8302 - accuracy: 0.7587 - val_loss: nan - val_accuracy: 0.7998
Epoch 4/10
108/108 [==============================] - 5s 46ms/step - loss: 0.5797 - accuracy: 0.8296 - val_loss: nan - val_accuracy: 0.8627
Epoch 5/10
108/108 [==============================] - 5s 44ms/step - loss: 0.3719 - accuracy: 0.8923 - val_loss: nan - val_accuracy: 0.9131
Epoch 6/10
108/108 [==============================] - 5s 44ms/step - loss: 0.2514 - accuracy: 0.9298 - val_loss: nan - val_accuracy: 0.9369
Epoch 7/10
108/108 [==============================] - 5s 44ms/step - loss: 0.1906 - accuracy: 0.9467 - val_loss: nan - val_accuracy: 0.9513


## Prediction (IMPLEMENTATION)

In [27]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape, y.shape[1], len(x_tk.word_index)+1, len(y_tk.word_index)+1)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

1/1 [==============================] - 1s 1s/step
Sample 1:
apprécié apprécié apprécié avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons avons gros
Il a vu un vieux camion jaune
Sample 2:
petits gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros gros
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [28]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['Traceback (most recent call last):',
 '  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>',
 '    sys.exit(main())',
 '             ^^^^^^',
 '  File "/usr/local/lib/python3.11/dist-packages/jupyter_core/application.py", line 283, in launch_instance',
 '    super().launch_instance(argv=argv, **kwargs)',
 '  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance',
 '    app.start()',
 '  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 420, in start',
 '    self.convert_notebooks()',
 '  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 582, in convert_notebooks',
 '    raise ValueError(msg)',
 "ValueError: Please specify an output format with '--to <format>'.",
 "The following formats are available: ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'qtpdf', 'qtpng', 'rst', 'script', 'slides', 'webpdf']"]

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?